In [63]:
#This code will get the list of prerequisites needed per class. This code is currently only for electrical eningeering and computer engineering.
#But can most likely be reused in future scrapes. 
from bs4 import BeautifulSoup
import requests
import pandas as pd
from queue import PriorityQueue

In [4]:
#url for the EEC and ECE catalogs and base url

EEC_url = "https://catalog.ucdavis.edu/departments-programs-degrees/electrical-computer-engineering/electrical-engineering-bs/?_gl=1*10xd7u3*_ga*MjgwNTkxMTcxLjE3NDM5NzQ2NTg.*_ga_2N9JP0W2PC*czE3NTE5NTEyNzckbzM0JGcxJHQxNzUxOTUxMzMyJGo1JGwwJGgw#requirementstext"
ECE_url = "https://catalog.ucdavis.edu/departments-programs-degrees/electrical-computer-engineering/computer-engineering-bs/#requirementstext"
base_url = "https://catalog.ucdavis.edu"

In [ ]:
#Getting the EEC Info

EEC_page = requests.get(EEC_url)
EEC_soup = BeautifulSoup(EEC_page.text, 'html')


#Getting all the class links
table = EEC_soup.find('table')
EEC_classes = table.find_all('a', class_='bubblelink code')

urls = [base_url + a['href'] for a in EEC_classes if 'href' in a.attrs]
# print(urls)


['https://catalog.ucdavis.edu/search/?P=CMN%20001', 'https://catalog.ucdavis.edu/search/?P=CMN%20001V', 'https://catalog.ucdavis.edu/search/?P=ENG%20003', 'https://catalog.ucdavis.edu/search/?P=ENG%20003Y', 'https://catalog.ucdavis.edu/search/?P=MAT%20021A', 'https://catalog.ucdavis.edu/search/?P=MAT%20021B', 'https://catalog.ucdavis.edu/search/?P=MAT%20021C', 'https://catalog.ucdavis.edu/search/?P=MAT%20021D', 'https://catalog.ucdavis.edu/search/?P=MAT%20022A', 'https://catalog.ucdavis.edu/search/?P=MAT%20022B', 'https://catalog.ucdavis.edu/search/?P=PHY%20009A', 'https://catalog.ucdavis.edu/search/?P=PHY%20009B', 'https://catalog.ucdavis.edu/search/?P=PHY%20009C', 'https://catalog.ucdavis.edu/search/?P=PHY%20009D', 'https://catalog.ucdavis.edu/search/?P=CHE%20002A', 'https://catalog.ucdavis.edu/search/?P=ENG%20006', 'https://catalog.ucdavis.edu/search/?P=ENG%20017', 'https://catalog.ucdavis.edu/search/?P=ENG%20017V', 'https://catalog.ucdavis.edu/search/?P=EEC%20001', 'https://catalog

In [61]:
def get_prereq_links(url):

    prerequisites_names_links = {}

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    first_section = block.find_all('p')

    # print(first_section)

    if len(first_section) == 1:

        return None
    
    elif len(first_section) == 2:

        prereq_description = first_section[1]
        
        if 'href' not in str(prereq_description):

            return None
        
        else:

            prereq_dirty = first_section[1].find_all('a')
            prereq_urls = [base_url + a['href'] for a in prereq_dirty if 'href' in a.attrs]

            # print(prereq_urls)
            
            for url in prereq_urls:

                page = requests.get(url)

                soup = BeautifulSoup(page.text, 'html')

                block = soup.find('div', class_ = 'courseblock')

                titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]

                prerequisites_names_links[titles[0]] = url

          
            




    return prerequisites_names_links





    

    

    

    


In [77]:
full_class_dictionary = {} #This contains every class and link to that class
course_and_prerequisites = {} #This contains every class and their list of prerequisites


class_queue = PriorityQueue()

already_seen = set()

for url in urls:
    class_queue.put(url)


while class_queue.empty() is False:

    url = class_queue.get()

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]

    if titles[0] not in already_seen:
       
       already_seen.add(titles[0])

       print(f'This is the current class: {titles[0]}')

    #    print(f'This is the queue size: {class_queue.qsize()}')

       full_class_dictionary[titles[0]] = url

       course_and_prerequisites[titles[0]] = get_prereq_links(url)

       if course_and_prerequisites[titles[0]] is not None:
            for urs in course_and_prerequisites[titles[0]].values():
                # print(f'Putting: {urs} in queue')
                class_queue.put(urs)



    # print(f'These are the prerequisites: {course_and_prerequisites[titles[0]]}')

    




print(full_class_dictionary)



This is the current class: CHE 002A
This is the current class: CHE 001
This is the current class: CHE 001V
This is the current class: CMN 001
This is the current class: CMN 001V
This is the current class: COM 001
This is the current class: COM 002
This is the current class: COM 003
This is the current class: COM 004
This is the current class: EEC 001
This is the current class: EEC 007
This is the current class: EEC 010
This is the current class: EEC 018
This is the current class: EEC 100
This is the current class: EEC 110A
This is the current class: EEC 111
This is the current class: EEC 130A
This is the current class: EEC 140A
This is the current class: EEC 140AV
This is the current class: EEC 150
This is the current class: EEC 161
This is the current class: EEC 196
This is the current class: ENG 003
This is the current class: ENG 003Y
This is the current class: ENG 006
This is the current class: ENG 017
This is the current class: ENG 017V
This is the current class: ENG 160
This is th

In [1]:
print(full_class_dictionary)

NameError: name 'full_class_dictionary' is not defined

In [ ]:
df = pd.DataFrame(columns= ['Course', 'Title', 'Units', 'Course Description', 'Prerequisites', 'Learning Activites', 'Credit Limitations', 'Grade Mode', 'General Education'])


for url in urls:

    #print(url)

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]
    #print(titles)

    first_section = block.find_all('p')
    second_section = block.find_all('li')

    #print(second_section)

    if len(first_section) == 1:

        course_description = first_section[0].get_text(strip=True).replace('Course Description:', '').strip()
        #print(course_description)

        prereq_description = None

    elif len(first_section) == 2:

        course_description = first_section[0].get_text(strip=True).replace('Course Description:', '').strip()
        #print(course_description)

        prereq_description = first_section[1].get_text(strip=True).replace('Prerequisite(s):', '').strip()
        




    learning_activities = second_section[0].get_text(strip=True).replace('Learning Activities:', '').strip()


    if len(second_section) == 3:

        credit_limitations = None


        grade_mode = second_section[1].get_text(strip=True).replace('Grade Mode:', '').strip()


        
        gen_ed = second_section[2].get_text(strip=True).replace('General Education:', '').strip()

    elif len(second_section) == 4:

        credit_limitations = second_section[1].get_text(strip=True).replace('Credit Limitation(s):', '').strip()


        grade_mode = second_section[2].get_text(strip=True).replace('Grade Mode:', '').strip()


        
        gen_ed = second_section[3].get_text(strip=True).replace('General Education:', '').strip()



    full_data = titles + [course_description] + [prereq_description] + [learning_activities] + [credit_limitations] + [grade_mode] + [gen_ed]
    #print(full_data)

    length = len(df)
    df.loc[length] = full_data

Completion of Entry Level Writing Requirement (ELWR).
Completion of Entry Level Writing Requirement (ELWR).
Two years of high school algebra, plane geometry, plane trigonometry, and analytical geometry, and satisfying the Mathematics Placement Requirement.
(MAT 021AC- or better orMAT 021AHC- or better) orMAT 017AB or better.
MAT 016C C- or better orMAT 017CC- or better orMAT 021BC- or better orMAT 021BHC- or better orMAT 017BB or better.
(MAT 021CC- or better orMAT 021CHC- or better) orMAT 017CB or better.
(MAT 016C C- or better orMAT 017CC- or better orMAT 021CC- or better orMAT 021CHC- or better); (ENG 006or EME 005 orECH 060orMAT 022AL(can be concurrent)).
(MAT 022AC- or better orMAT 027AC- or better orBIS 027AC- or better orMAT 067C- or better).


KeyboardInterrupt: 

In [ ]:
#Data base should sort the classes from most prerequisite history to least. This only applies to classes, not special cases like "taken highscool math"
#New Data base will contain all the previous data base into, plus # of prerequisites, and the class description of those classes will be added as well. Also the class link will be added